<a href="https://colab.research.google.com/github/SatLight/Airline-Delay-Prediction-using-Spark-and-Kylin/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import findspark
findspark.init()
import pyspark

from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DoubleType
from pyspark.sql.functions import udf
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Decision Tree") \
    .getOrCreate()

In [ ]:
df = spark.read.option("header", True).csv("file:///home/sanjithsjadhav/programming/java/sem5/hadoop/project/2016.csv")

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
#  OP_CARRIER_FL_NUM  DISTANCE    ACTUAL_ELAPSED_TIME
columns_to_drop = ['OP_CARRIER','TAXI_OUT','CRS_ELAPSED_TIME','DEP_TIME','WHEELS_OFF',
             'AIR_TIME','WHEELS_ON','TAXI_IN','ARR_TIME','CANCELLATION_CODE']
df = df.drop(*columns_to_drop)
# len(columns_to_drop)

In [ ]:
len(df.columns)

In [ ]:
df = df.drop(df.columns[17])

In [ ]:
df.printSchema()

In [ ]:
len(df.columns)

In [ ]:
# df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()
df.select([count(when(isnan(c) | col(c).isNull() , True)) for c in df.columns]).show()

In [ ]:
columns_with_nas = ['DEP_DELAY', 'ARR_DELAY', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'ACTUAL_ELAPSED_TIME']
removeAllDF = df.na.drop()
for x in columns_with_nas:
    meanValue = removeAllDF.agg(avg(x)).first()[0]
#     print(x, meanValue)
    df= df.na.fill({x : meanValue})

In [ ]:
df = df.withColumn('RESULT', df.ARR_DELAY>15)

In [ ]:
df.head(5)

In [ ]:
test = df.select([col(c).cast("integer") for c in ["RESULT"]])
# df = df.withColumn('result', test['RESULT'])
df = df.withColumn( "RESULT" , F.when( F.col("RESULT") , F.lit(1) ).otherwise(0) )

In [ ]:
df.head(5)

In [ ]:
# df.select([count(when(isnan(df.ARR_DELAY) | col(df.ARR_DELAY).isNull() , True))]).show()
list(map(lambda c : (c,df.filter(df[c].isNull()).count()), columns_with_nas))

In [ ]:
list(map(lambda c : (c,removeAllDF.filter(removeAllDF[c].isNull()).count()), columns_with_nas))

In [ ]:
df.groupBy('RESULT').count().show()

In [ ]:
indexes = ['ORIGIN','DEST','ARR_DELAY','FL_DATE']
# df = df.drop(*columns_to_drop)
labelIndexer = [StringIndexer(inputCol=column, outputCol=column+'_index').fit(df) for column in indexes]
labelIndexer

[StringIndexerModel: uid=StringIndexer_39f2214a345e, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_987cfdcd3ab5, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_d8922b18fa5e, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_08059f6ef9f0, handleInvalid=error]

In [ ]:
df.columns

['FL_DATE',
 'OP_CARRIER_FL_NUM',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'DEP_DELAY',
 'CRS_ARR_TIME',
 'ARR_DELAY',
 'CANCELLED',
 'DIVERTED',
 'ACTUAL_ELAPSED_TIME',
 'DISTANCE',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'RESULT']

In [ ]:
c = list(df.columns)[:-1]
for cols in c[:1]+c[2:]:
    df = df.withColumn(cols, df[cols].cast(DoubleType()))

In [ ]:
df.printSchema()

root
 |-- OP_CARRIER_FL_NUM: double (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- CRS_DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- CRS_ARR_TIME: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)
 |-- SECURITY_DELAY: double (nullable = true)
 |-- LATE_AIRCRAFT_DELAY: double (nullable = true)
 |-- RESULT: integer (nullable = false)



In [ ]:
len(df.columns)

In [ ]:
cols = ['OP_CARRIER_FL_NUM','ACTUAL_ELAPSED_TIME','DISTANCE','CRS_DEP_TIME','DEP_DELAY','CRS_ARR_TIME','CANCELLED','DIVERTED','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']
assembler = VectorAssembler(inputCols=cols,outputCol="features")
output = assembler.transform(df)
output = output.drop(*cols)
output.head(5)

IllegalArgumentException: Data type string of column OP_CARRIER_FL_NUM is not supported.

In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scalerModel = scaler.fit(output)
output = scalerModel.transform(output)
output = output.drop('features')
output.head(5)

[Row(ORIGIN='DTW', RESULT=0, scaled_features=DenseVector([0.7339, 3.7928, 3.1945, 3.9523, 0.0, 4.1527, 0.0, 0.0, 0.8527, 0.3049, 1.1598, 0.0848, 1.2776])),
 Row(ORIGIN='ATL', RESULT=0, scaled_features=DenseVector([0.7357, 1.4506, 1.0331, 4.3404, 0.1254, 4.4956, 0.0, 0.0, 0.8527, 0.3049, 1.1598, 0.0848, 1.2776])),
 Row(ORIGIN='LAX', RESULT=0, scaled_features=DenseVector([0.7375, 3.0742, 3.1429, 4.6059, 0.0251, 1.1621, 0.0, 0.0, 0.8527, 0.3049, 1.1598, 0.0848, 1.2776])),
 Row(ORIGIN='SLC', RESULT=0, scaled_features=DenseVector([0.7381, 2.5685, 2.5666, 3.3825, 0.1003, 4.3174, 0.0, 0.0, 0.8527, 0.3049, 1.1598, 0.0848, 1.2776])),
 Row(ORIGIN='BZN', RESULT=1, scaled_features=DenseVector([0.7386, 2.5019, 1.4108, 1.8383, 1.8055, 2.3553, 0.0, 0.0, 3.0308, 0.0, 4.1039, 0.0, 0.0]))]

In [ ]:
dt = DecisionTreeClassifier(labelCol="RESULT", featuresCol="scaled_features")

In [ ]:
pipeline = Pipeline(stages=[labelIndexer,dt])

In [ ]:
test, train = output.randomSplit([0.7, 0.3])

In [ ]:
model = pipeline.fit(train)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.select("prediction", "RESULT", "scaled_features").show(5)

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="RESULT", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions)
accuracy

0.9955288147850174

In [ ]:
treeModel = model.stages[0]
treeModel

In [ ]:
t = test.select([col(c).cast("integer") for c in ["RESULT"]])

In [ ]:
test.head(5)

In [ ]:
output = output.withColumn('RESULT', t.RESULT)

In [ ]:
output.head(10)